# 소프트맥스(다중 분류 로지스틱 회귀 모델)
M개의 입력을 받아 N개의 클래스로 출력하는 로지스틱 회귀 모델을 케라스로 구현해본다.  
보통 다중 분류 로지스틱 회귀 모델을 소프트맥스(softmax)라고 부른다.  
이번 예제에서는 케라스에서 제공하는 MNIST 손글씨 숫자 데이터셋을 사용해 입력된 손글씨 숫자를 0부터 9까지로 분류해본다.

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras.datasets import mnist

## 데이터 획득
MNIST 손글씨 데이터를 내려받아 변수에 저장한다.

In [20]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


손글씨 데이터(X_train, X_test)가 가로 28픽셀, 세로 28픽셀로 구성된 것을 확인할 수 있다.  
학습에 사용될 X_train은 총 60000개의 데이터, 테스트에 사용될 X_test는 총 10000개의 데이터가 있다.

In [21]:
print('train data (count, row, column) : ' + str(X_train.shape))
print('test data (count, row, column) : ' + str(X_test.shape))

train data (count, row, column) : (60000, 28, 28)
test data (count, row, column) : (10000, 28, 28)


학습 데이터 중 하나의 샘플을 출력해보자.  
각각의 픽셀은 0부터 255까지의 값을 가지고 있다.

In [22]:
print(X_train[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

## 데이터 정규화
모델 학습 시작에 앞서 데이터를 정규화한다.  
정규화는 입력값을 0부터 1의 값으로 변경하게 된다.  
정규화된 입력값은 경사하강법으로 모델 학습 시, 보다 쉽고 빠르게 최적의 w,b를 찾도록 도와준다

In [24]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

아래 명령어를 통해 정규화된 데이터를 확인할 수 있다.

In [25]:
print(X_train[0])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

y_train, y_test는 손글씨 데이터(28*28 픽셀 데이터)에 해당하는 숫자를 나타낸다.  
y_train은 총 6만개, y_test는 총 1만개의 숫자를 가지고 있다.  

In [26]:
print('train target (count) : ' + str(y_train.shape))
print('test target (count) : ' + str(y_test.shape))

train target (count) : (60000,)
test target (count) : (10000,)


아래 코드를 실행하여, y_train, y_test에서 샘플로 숫자를 출력해본다.

In [27]:
print('sample from train: ' + str(y_train[0]))
print('sample from test: ' + str(y_test[0]))

sample from train: 5
sample from test: 7


## 데이터 단순화
이번 실습에서는 28*28 픽셀의 행/열 지역적인 정보를 사용하지 않고 단순히 정규화된 입력값만 가지고 숫자를 분류할 것이기 때문에  
행과 열의 구분 없이 단순히 784(28*28) 길이의 배열로 데이터를 단순화한다.

In [29]:
input_dim = 784
X_train = X_train.reshape(60000, input_dim)
X_test = X_test.reshape(10000, input_dim)

In [30]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


# 소프트맥스
소프트맥스는 정규화된 여러 개의 로지스틱 회귀로 구성되어 있으며, 10개의 로지스틱 회귀를 배열로 나타낼 경우  
[L0, L1, L2, L3, L4, L5, L6, L7, L8, L9]로 나타낼 수 있다.  
로지스틱 회귀이므로 각 L의 값은 0부터 1이며, 만약 출력이 [0.9, 0.1, 0, 0, 0, 0, 0, 0, 0, 0]일 경우 0번째 인덱스가 소프트맥스의 출력값이 된다.  
학습 시, y값(실제값)과의 cross entropy를 측정해야 하므로, 아래의 코드를 실행하여 y를 one hot encoding으로 변환시켜준다.

In [31]:
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

아래 코드를 실행하면 5였던 값이 one hot encoding으로 변환되어 클래스 갯수만큼의 길이를 갖는 벡터로 변경 되었고, 5에 해당 하는 인덱스의 값이 1인 것을 확인할 수 있다.

In [32]:
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


케라스의 Sequential()을 사용하면 간단하게 소프트맥스를 구현할 수 있다.  
총 784개(28*28)의 입력을 받아 10개의 시그모이드 값을 출력하는 모델을 다음 코드로 구현한다.

In [33]:
model = Sequential()
model.add(Dense(input_dim=input_dim, units=10, activation='softmax'))

## 모델 학습 
10개의 클래스로 분류할 것이기 때문에 categorical_crossentropy를 비용함수로 사용한 경사하강법으로  
최적의 회귀계수(w)와 편향(biases)를 구한다.  

In [34]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=2048, epochs=10, verbose=0)

## 모델 테스트

In [37]:
score = model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

313/313 [==============================] - 0s 516us/step - loss: 1.0027 - accuracy: 0.8071
Test accuracy: 0.8070999979972839


## 모델 요약
총 10개의 로지스틱 회귀가 있고, 각 로지스틱 회귀는 784개의 weights와 1개의 bias를 가지고 있기 때문에,  
총 7850(785*10)개의 param이 있는 것을 볼 수 있다.

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


회귀계수의 개수와 편향의 개수는 다음 명령어로 확인할 수 있다.  
첫번째 레이어에 존재하는 w1,w2,...,w784, b1,b2,...b10은 아래와 같다.

In [39]:
model.layers[0].weights

[<tf.Variable 'dense/kernel:0' shape=(784, 10) dtype=float32, numpy=
 array([[-0.03159061,  0.08117586,  0.07849469, ...,  0.07572704,
          0.03545845, -0.04853053],
        [ 0.08691143,  0.08393691,  0.07244744, ...,  0.02268288,
         -0.08291754,  0.03731216],
        [ 0.06826929, -0.00379188, -0.07427898, ..., -0.0195033 ,
          0.04518972,  0.07728856],
        ...,
        [-0.07830705,  0.00193922,  0.00236508, ...,  0.03020824,
          0.04791834,  0.06497041],
        [ 0.02620923, -0.0108872 ,  0.05721875, ...,  0.04062779,
          0.03470246,  0.02095   ],
        [ 0.02325033,  0.05918721, -0.02016705, ...,  0.04050238,
         -0.05611946, -0.0526439 ]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=
 array([-0.0315285 ,  0.06883556, -0.02048932, -0.02745991,  0.02053321,
         0.01387148, -0.0027065 ,  0.02752863, -0.04882486,  0.00024025],
       dtype=float32)>]